In [1]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    
    scores = []

    a_start,a_step,a_num = 0.00100,0.0000095,150

    params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

    if 1: 
        pool = Pool(processes=1)
        ls   = pool.map( one_loop1, params )


    if 0:
        ls = [ one_loop(param) for param in  params ]


    ##------------------------------------------------

    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels
var1 = 0
def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     if var1 != 0:
#         df = submission.merge(hits,  on=['hit_id'], how='left')
#     else:
#         df = hits
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

#         if len(df1) == 0:
#             continue
        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

#         print(df1.head())
        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
# rz_scales=[0.65, 0.965, 1.528]
# def _eliminate_outliers(labels,M):
#         norms=np.zeros((len(labels)),np.float32)
#         indices=np.zeros((len(labels)),np.float32)
#         for i, cluster in tqdm(enumerate(labels),total=len(labels)):
#             if cluster == 0:
#                 continue
#             index = np.argwhere(self.clusters==cluster)
#             index = np.reshape(index,(index.shape[0]))
#             indices[i] = len(index)
#             x = M[index]
#             norms[i] = self._test_quadric(x)
#         threshold1 = np.percentile(norms,90)*5
#         threshold2 = 25
#         threshold3 = 6
#         for i, cluster in enumerate(labels):
#             if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
#                 self.clusters[self.clusters==cluster]=0

# def _test_quadric(x):
#     if x.size == 0 or len(x.shape)<2:
#         return 0
#     Z = np.zeros((x.shape[0],10), np.float32)
#     Z[:,0] = x[:,0]**2
#     Z[:,1] = 2*x[:,0]*x[:,1]
#     Z[:,2] = 2*x[:,0]*x[:,2]
#     Z[:,3] = 2*x[:,0]
#     Z[:,4] = x[:,1]**2
#     Z[:,5] = 2*x[:,1]*x[:,2]
#     Z[:,6] = 2*x[:,1]
#     Z[:,7] = x[:,2]**2
#     Z[:,8] = 2*x[:,2]
#     Z[:,9] = 1
#     v, s, t = np.linalg.svd(Z,full_matrices=False)        
#     smallest_index = np.argmin(np.array(s))
#     T = np.array(t)
#     T = T[smallest_index,:]        
#     norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
#     return norm

# def _preprocess(hits):
#     x = hits.x.values
#     y = hits.y.values
#     z = hits.z.values

#     r = np.sqrt(x**2 + y**2 + z**2)
#     hits['x2'] = x/r
#     hits['y2'] = y/r

#     r = np.sqrt(x**2 + y**2)
#     hits['z2'] = z/r

#     ss = StandardScaler()
#     X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

#     return X

In [7]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.2691
    w6 = 0.0020
    w6 = [0.0000001, 1.2] 
    w10 = [0.00001, 1.2] 
    w12 = [0.00001, 0.1] 
    Niter = 179
    w12 = 0.00045
#     Fun4BO21(w12)
#     for w12 in np.arange(0.0002, 0.003, 0.00005):
#         print(w12)
#         Fun4BO21(w12)
#     bo = BayesianOptimization(Fun4BO21,pbounds = {'w12':w12})
#     bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

# z1+z2: 13 | 05m39s |    0.55616 |    0.1124 | 
# z1 * z2: 13 | 05m40s |    0.55637 |    0.0404 | 
# tt = cos(theta), theta = np.arctan2(rt,z): 4 | 06m01s |    0.55196 |    0.2711 | 
# tt, theta = np.arctan2(rt2,z2);  8 | 05m39s |    0.55604 |    0.0005 | 
# cos + sin: 15 | 05m39s |    0.55714 |    0.2691 | 
# cos-sin: 9 | 05m51s |    0.55714 |    0.0020 | 
# cos+sin/cos-sin: 8 | 06m03s |    0.55694 |    0.0012 | 
# ctt, stt, ctt+stt/ctt-stt: 8 | 06m03s |    0.55273 |    0.0000 | 
# ctt: 10 | 05m39s |    0.55613 |    0.0047 | 
# caa * stt: 7 | 05m38s |    0.55613 |    0.0022 |
# saa * ctt: 15 | 05m38s |    0.55613 |    0.0033 |
# c1 = caa*stt, c2 = saa*ctt, c1+c2: 10 | 05m43s |    0.55622 |    0.0571 | 
# caa * saa: 7 | 05m45s |    0.55639 |    0.1548 | 
# caa/saa: 13 | 05m43s |    0.55613 |    0.000001 | 
# xx1, xx2: 12 | 06m04s |    0.55613 |    0.000001 |    0.000001 | 

# hdbscan - braycurtis: 0.14424123982821765
# hdbscan - euclidean : 0.1311

# eps: 0.0040: 0.57549
# tt: 21 | 07m08s |    0.57208 |    0.0000 |  (1e-07)
# ctt, stt (after new eqn: 25 | 07m33s |    0.57254 |    0.0026 | 
# ctt, stt: 0.00261: 0.5727074941034839
# caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt, y1, y3:
# w1 = 1.1932215111905984, w2 = 0.39740553885387364, w3 = 0.3512647720585538, w4 = 0.1470, w5 = 0.01201
# w6 = 0.0205, w7 = 0.00261, w8 = 0.0049, w9 = 0.0012 (0.5717942069958433)
# 
# X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3])):
# w1 = 1.1932215111905984 w2 = 0.39740553885387364 w3 = 0.3512647720585538 w4 = 0.1470 w5 = 0.01201 w6 = 0.0003864 
# w7 = 0.0205 w8 = 0.0049 w9 = 0.00121 (0.57343)
# ctt, stt: 13 | 08m16s |    0.57343 |    0.0003 | (0.00032967312140735677)
# ctt, stt: 21 | 08m20s |    0.57343 |    0.0000 |  (1.4930496676654575e-05)
# ctt, stt: 15 | 08m14s |    0.57351 |    0.8435 | (t1 = theta+mm*(rt+ w11*rt**2)/1000*(ii/2)/180*np.pi)
# z4: 0.0245 (0.5735925042985041)
# # z4 0.0318 (0.5736635664313068)
# x4: 0.00001 (0.5735421714482896)
# x4: 0.00025 (0.5736999491677117)
# x4: 0.00045 (0.5737240529228144)

In [8]:
# def trackML32(w13):
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
#     w4 = 0.1470
#     w5 = 0.01201
#     w6 = 0.0003864
#     w7 = 0.0205
#     w8 = 0.0049
#     w9 = 0.00121
#     w10 = 1.4930496676654575e-05
#     w11 = 0.0318
#     w12 = 0.00045
#     Niter=179
#     Niter=247

#     df = hits
#     x  = df.x.values
#     y  = df.y.values
#     z  = df.z.values

#     dz = 0
#     z = z + dz

#     rt  = np.sqrt(x**2+y**2)
#     r  = np.sqrt(x**2+y**2+z**2)
#     a0  = np.arctan2(y,x)

#     x2 = x/r
#     y2 = y/r


#     phi = np.arctan2(y, x)
#     phi_deg= np.degrees(np.arctan2(y, x))


#     z1 = z/rt
#     z2 = z/r

#     z3 = np.log1p(abs(z/r))*np.sign(z)

#     x1 = x/rt
#     y1 = y/rt

#     y3 = np.log1p(abs(y/r))*np.sign(y)

#     theta = np.arctan2(rt, z)
#     theta_deg = np.degrees(np.arctan2(rt, z))
#     tt = np.tan(theta_deg)

#     z4 = np.sqrt(abs(z/rt))

#     x4 = np.sqrt(abs(x/r))
#     y4 = np.sqrt(abs(y/r))

#     mm = 1
#     ls = []

#     #     print(Niter)
#     #     Niter = Niter.astype(np.int32)
#     #     print(Niter)
#     #     print(w1, w2, w3, Niter)

#     for ii in range(Niter):
#         mm = mm * (-1)

#         a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
#     #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)

#         t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
#         ctt = np.cos(t1)
#         stt = np.sin(t1)
#         ttt = np.tan(t1)
#         deps = 0.0000025
#     #         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt]))
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
#                                                             ctt, stt, z4, x4]))
#     #         print(X.shape)

#     #         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10, w11, w12]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#     #         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,cluster_selection_method='leaf',metric='euclidean', leaf_size=50)
#     #         l = cl.fit_predict(X)+1

#         l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     #         l= DBSCAN(eps=w7,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)

# #     for ii in range(Niter):
# #         mm = mm * (1)

# #         a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
# #     #         print(a0, a1)
# #         saa = np.sin(a1)
# #         caa = np.cos(a1)

# #         t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
# #         ctt = np.cos(t1)
# #         stt = np.sin(t1)
# #         ttt = np.tan(t1)
# #         deps = 0.0000025
# #     #         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt]))
# #         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
# #                                                             ctt, stt, z4, x4]))
        
# #         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #     #         print(X.shape)

# #     #         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
# #         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10, w11, w12]
# #         cx = [w13,w13,w2,w3]
# #         for jj in range(X.shape[1]): 
# #              X[:,jj] = X[:,jj]*cx[jj]
# #         deps = 0.0000025
# #     #         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,cluster_selection_method='leaf',metric='euclidean', leaf_size=50)
# #     #         l = cl.fit_predict(X)+1

# #         l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
# #     #         l= DBSCAN(eps=w7,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
# #         ls.append(l)

#     X = _preprocess(hits) 
#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
#     labels = np.unique(ls)
#     self._eliminate_outliers(labels,X)          
#     max_len = np.max(self.clusters)
#     mask = self.clusters == 0
#     self.clusters[mask] = cl.fit_predict(X[mask])+max_len
#     return self.clusters
    
#     num_hits=len(df)
#     labels = np.zeros(num_hits,np.int32)
#     counts = np.zeros(num_hits,np.int32)
#     for l in ls:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)

#     track_id = labels
#     sum_score=0
#     sum = 0
#     submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
#         data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
#     ).astype(int)
#     for i in range(8):
#         submission = extend(submission,hits)
#         score = score_event(truth, submission)
#         print('[%2d] score : %0.8f'%(i, score))
#         sum_score += score
#         sum += 1

#     print('--------------------------------------')
#     sc = sum_score/sum
#     print(sc)
# # org score: 0.5737240529228144

In [9]:
# trackML32(1)

In [10]:
# from sklearn.preprocessing import StandardScaler
# import hdbscan
# from scipy import stats
# from tqdm import tqdm
# from sklearn.cluster import DBSCAN

# class Clusterer(object):
#     def __init__(self,rz_scales=[0.65, 0.965, 1.528]):                        
#         self.rz_scales=rz_scales
    
#     def _eliminate_outliers(self,labels,M):
#         norms=np.zeros((len(labels)),np.float32)
#         indices=np.zeros((len(labels)),np.float32)
#         for i, cluster in tqdm(enumerate(labels),total=len(labels)):
#             if cluster == 0:
#                 continue
#             index = np.argwhere(self.clusters==cluster)
#             index = np.reshape(index,(index.shape[0]))
#             indices[i] = len(index)
#             x = M[index]
#             norms[i] = self._test_quadric(x)
#         threshold1 = np.percentile(norms,90)*5
#         threshold2 = 25
#         threshold3 = 6
#         for i, cluster in enumerate(labels):
#             if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
#                 self.clusters[self.clusters==cluster]=0   
#     def _test_quadric(self,x):
#         if x.size == 0 or len(x.shape)<2:
#             return 0
#         Z = np.zeros((x.shape[0],10), np.float32)
#         Z[:,0] = x[:,0]**2
#         Z[:,1] = 2*x[:,0]*x[:,1]
#         Z[:,2] = 2*x[:,0]*x[:,2]
#         Z[:,3] = 2*x[:,0]
#         Z[:,4] = x[:,1]**2
#         Z[:,5] = 2*x[:,1]*x[:,2]
#         Z[:,6] = 2*x[:,1]
#         Z[:,7] = x[:,2]**2
#         Z[:,8] = 2*x[:,2]
#         Z[:,9] = 1
#         v, s, t = np.linalg.svd(Z,full_matrices=False)        
#         smallest_index = np.argmin(np.array(s))
#         T = np.array(t)
#         T = T[smallest_index,:]        
#         norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
#         return norm

#     def _preprocess(self, hits):
        
#         x = hits.x.values
#         y = hits.y.values
#         z = hits.z.values

#         r = np.sqrt(x**2 + y**2 + z**2)
#         hits['x2'] = x/r
#         hits['y2'] = y/r

#         r = np.sqrt(x**2 + y**2)
#         hits['z2'] = z/r

#         ss = StandardScaler()
#         X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#         for i, rz_scale in enumerate(self.rz_scales):
#             X[:,i] = X[:,i] * rz_scale
       
#         return X
#     def _init(self, dfh, w1, w2, w3, w4, w5, w6, w7, w8,w9,w10,w11,w12, epsilon, Niter, z_shift):
#         # z- shift
#         dfh['z'] = dfh['z'].values - z_shift
#         dfh['r'] = np.sqrt(dfh['x'].values ** 2 + dfh['y'].values ** 2 + dfh['z'].values ** 2)
#         dfh['rt'] = np.sqrt(dfh['x'].values ** 2 + dfh['y'].values ** 2)
#         dfh['a0'] = np.arctan2(dfh['y'].values, dfh['x'].values)
#         dfh['z1'] = dfh['z'].values / dfh['rt'].values
#         dfh['z2'] = dfh['z'].values / dfh['r'].values
#         dfh['s1'] = dfh['hit_id']
#         dfh['N1'] = 1
#         dfh['z1'] = dfh['z'].values / dfh['rt'].values
#         dfh['z2'] = dfh['z'].values / dfh['r'].values
#         dfh['x1'] = dfh['x'].values / dfh['y'].values
#         dfh['x2'] = dfh['x'].values / dfh['r'].values
#         dfh['x3'] = dfh['y'].values / dfh['r'].values
#         dfh['x4'] = dfh['rt'].values / dfh['r'].values
#         dfh['x5'] = dfh['y'].values / dfh['rt'].values
        
#         dfh['z3'] = np.log1p(abs((dfh['z'].values / dfh['r'].values)))*np.sign(dfh['z'])
#         dfh['y3'] = np.log1p(abs((dfh['y'].values / dfh['r'].values)))
        
#         dfh['z4'] = np.sqrt(abs((dfh['z'].values / dfh['rt'].values)))
        
#         dfh['x6'] = np.sqrt(abs((dfh['x'].values / dfh['r'].values)))
        
#         dfh['theta'] = np.arctan2(dfh['rt'].values, dfh['z'].values)
#         dfh['theta_deg'] = np.degrees(np.arctan2(dfh['rt'].values, dfh['z'].values))
    
#         mm = 1
# #         print(dfh['s1'].values)
#         for ii in tqdm(range(int(Niter))):
#             mm = mm * (-1)
#             dfh['a1'] = dfh['a0'].values + mm * (dfh['rt'].values + 0.0000145
#                                                  * dfh['rt'].values ** 2) / 1000 * (ii / 2) / 180 * np.pi
#             dfh['sina1'] = np.sin(dfh['a1'].values)
#             dfh['cosa1'] = np.cos(dfh['a1'].values)
#             ss = StandardScaler()
# #             dfs = ss.fit_transform(dfh[['sina1', 'cosa1', 'z1', 'z2','x1','x2','x3','x4']].values)
# #             cx = np.array([w1, w1, w2, w3, w4, w5, w6, w7])
#             dfh['t1'] = dfh['theta'].values+mm*(dfh['rt'].values+ 0.8435*dfh['rt'].values**2)/1000*(ii/2)/180*np.pi
#             dfh['stt'] = np.sin(dfh['t1'].values)
#             dfh['ctt'] = np.cos(dfh['t1'].values)
        
# #             X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
# #                                                             ctt, stt, z4, x4]))
            
#             dfs = ss.fit_transform(dfh[['sina1', 'cosa1', 'z1', 'z2', 'x4', 'x2', 'x3', 'z3', 'x5', 'y3', 
#                                                             'z4', 'x6']])
#     #         print(X.shape)

#     #         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
#             cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w11, w12]
#             dfs = np.multiply(dfs, cx)
#             clusters = DBSCAN(eps=epsilon, min_samples=1, metric="euclidean", n_jobs=32).fit(dfs).labels_
#             dfh['s2'] = clusters
#             dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
#             maxs1 = dfh['s1'].max()
#             dfh.loc[(dfh['N2'] > dfh['N1']) & (dfh['N2'] < 20),'s1'] = dfh['s2'] + maxs1
#             dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
# #         print(dfh['s1'].values)
#         return dfh['s1'].values
#     def predict(self, hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12, epsilon, Niter, z_shift):         
# #         self.clusters = self._init(hits,2.7474448671796874,1.3649721713529086,0.7034918842926337,
# #                                         0.0005549122352940002,0.023096034747190672,0.04619756315527515,
# #                                         0.2437077420144654,0.009750302717746615,338)

        
        
#         self.clusters = self._init(hits,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,epsilon,Niter, z_shift)

                                       
# #         X = self._preprocess(hits) 
# #         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
# #                              metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
# #         labels = np.unique(self.clusters)
# #         self._eliminate_outliers(labels,X)          
# #         max_len = np.max(self.clusters)
# #         mask = self.clusters == 0
# #         print(X[mask].shape)
# #         self.clusters[mask] = cl.fit_predict(X[mask])+max_len
#         return self.clusters

In [13]:
# # w1 = 1.1932215111905984
# w1 = 1.124
# # w2 = 0.39740553885387364
# w2 = 0.575
# # w3 = 0.3512647720585538
# w3 = 0.125
# # w4 = 0.1470
# w4 = 0.165
# # w5 = 0.01201
# w5 = 0.015
# # w6 = 0.0003864
# w6 = 0.0003
# # w7 = 0.0205
# w7 = 0.025
# w8 = 0.0049
# # w9 = 0.00121
# w9 = 0.0015
# w10 = 1.4930496676654575e-05
# # w11 = 0.0318
# w11 = 0.015
# # w12 = 0.00045
# w12 = 0.00065
# Niter = 247
# epsilon = 0.005 # after iteration
# z_shift = 55
# # for w1 in np.arange(1.0,2.0,0.1):
# model = Clusterer()
# labels = model.predict(hits,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,epsilon,Niter, z_shift)
# submission = create_one_event_submission(0, hits, labels)
# score = score_event(truth, submission)
# print(w1, score)
# # w1: 1.124 - 0.5279192227189358
# # w2: 0.575: 0.5293042848848362
# # w3: 0.125: 0.5384771485839126
# # w4: 0.165: 0.5387405251950346
# # w5: 0.015: 0.5397604313593793
# # w6: 0.0003: 0.5397817800394703
# # w7: 0.025: 0.539861997939812
# # w8: 0.0049: 0.539861997939812
# # w9: 0.0015 0.5398880399799231
# # w10: 0.0015: 0.5398843174199072 (dont use)
# # w11: 0.015: 0.5399689874402678
# # w12: 0.00065 0.5400076261804324
# # epsilon 0.005: 0.541690617197602 ( with stuff commented out)
# # with z_shift 5.5: 0.47561716960612915 
# # with z_shift 55: 0.01741897458420483

100%|██████████| 247/247 [05:59<00:00,  1.46s/it]


1.124 9.897752042164421e-05


In [12]:
# submission = create_one_event_submission(0, hits, labels)
# score = score_event(truth, submission)
# print("Your score: ", score)
# # kernel org: 0.5129067228149826
# # my org: 0.4321907182011324
# # epsilon (0.00975, 0.33325577687966956)

Your score:  0.0006367412927125178


In [1]:
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from sklearn.cluster.dbscan_ import dbscan
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import timeit
import multiprocessing
from multiprocessing import Pool
import random

In [2]:
np.random.seed(123)

In [19]:
def make_counts(labels):
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts
def find_labels1(params):
    hits, dz, eps = params
    a = hits['phi'].values
    z = hits['z'].values
    zr = hits['zr'].values
    zrt = hits['zrt'].values
    aa = a + np.sign(z) * dz * z

    f0 = np.cos(aa)
    f1 = np.sin(aa)
    f2 = zr
    f3 = zrt
    X = StandardScaler().fit_transform(np.column_stack([f0, f1, f3]))

def find_labels(params):
    hits, dz, eps, ii = params
    a0 = hits['phi'].values
    z = hits['z'].values
    r = hits['zr'].values
    rt = hits['zrt'].values
#     aa = a + np.sign(z) * dz * z

    a1 = a0 - (rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
    
    f0 = np.cos(a1)
    f1 = np.sin(a1)
    f2 = r
    f3 = rt
    X = StandardScaler().fit_transform(np.column_stack([f0, f1, f3]))

    
    
#     _, l = dbscan(X, eps=0.0048, min_samples=1, n_jobs=8)
    _, l = dbscan(X, eps=eps, min_samples=1, n_jobs=8)
    return l

def add_count(l):
    unique, reverse, count = np.unique(l, return_counts=True, return_inverse=True)
    c = count[reverse]
    c[np.where(l == 0)] = 0
    c[np.where(c > 20)] = 0
    return (l, c)

def do_dbscan_predict(hits, truth, z_s, eps):
    start_time = timeit.default_timer()
    params = []
#     shifts = list(np.random.uniform(low=-5.5, high=5.5, size=(50,)))
#     eps = 0.0048 + z_s*10**(-5)
    hits['z'] = hits['z'].values + z_s
    hits['rt'] = np.sqrt(hits['x'] ** 2 + hits['y'] ** 2)
    hits['r'] = np.sqrt(hits['x'] ** 2 + hits['y'] ** 2 + hits['z']**2)
    hits['zr'] = hits['z'] / hits['r']
    hits['zrt'] = hits['z'] / hits['rt']
    hits['phi'] = np.arctan2(hits['y'], hits['x'])
    ii = 0
    for i in range(0, 20):
        dz = i * 0.00001
        params.append((hits, dz, eps, ii))
        if i > 0:
             params.append((hits, -dz, eps, ii))
        ii += 1
    # Kernel time is limited. So we skip some angles.
    for i in range(20, 180):
        dz = i * 0.00001
        if i % 2 == 0:
            params.append((hits, dz, eps,ii))
        else:
             params.append((hits, -dz, eps,ii))
        ii += 1
#     print(len(params))
    pool = Pool(processes=8)
    labels_for_all_steps = pool.map(find_labels, params)
    results = [add_count(l) for l in labels_for_all_steps]
    pool.close()

# for l in ls:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)
        
    labels, counts = results[0]
    for i in range(1, len(results)):
        l, c = results[i]
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts[idx] = c[idx]

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    score = score_event(truth, submission)
    print('[%5f], z_s: %0.8f, score : %0.8f' % (timeit.default_timer() - start_time, z_s, score))

    return labels

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

def run_dbscan():
    data_dir = '../data/train'

    event_ids = ['000001030']
    sum = 0
    sum_score = 0
    
    for i, event_id in enumerate(event_ids):
        hits, cells, particles, truth = load_event(data_dir + '/event' + event_id)
        num_hits = len(hits)
        shifts = list(np.linspace(-5.5, 5.5, 50))
        ls = []
        labels = do_dbscan_predict(hits, truth, 0, 0.0048)
        
        submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f' % (i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    print(sum_score / sum)
    
def run_dbscan1():
    data_dir = '../data/train'

    event_ids = ['000001030']
    sum = 0
    sum_score = 0
    
    for i, event_id in enumerate(event_ids):
        hits, cells, particles, truth = load_event(data_dir + '/event' + event_id)
        num_hits = len(hits)
        shifts = list(np.linspace(-5.5, 5.5, 50))
        ls = []
        labels = do_dbscan_predict(hits, truth, 0, 0.0048)
        ls.append(labels)
        
        
        labels1 = labels
        counts1 = make_counts(labels1)
        
#         shifts = [-5.5, -5.27551020, -5.05102041, 5.05102041]
#         epss = [0.0045, 0.0046, 0.0047]
        for z_s in shifts:
            e = 0.0048
#             for e in epss:
            print(z_s, e)
            hits, cells, particles, truth = load_event(data_dir + '/event' + event_id)
            num_hits = len(hits)
            labels = do_dbscan_predict(hits, truth, z_s, e)
            ls.append(labels)
            c = make_counts(labels)
            idx = np.where((c-counts1>0) & (c<20))[0]
            labels1[idx] = labels[idx] + labels1.max()
            counts1 = make_counts(labels1)

            submission = create_one_event_submission(0, hits['hit_id'].values, labels1)
            score = score_event(truth, submission)
            print('[%.8f] score : %0.8f' % (z_s, score))
        
#         num_hits = len(hits)
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
    
        for l in ls:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)
        submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f' % (i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    print(sum_score / sum)


In [20]:

if __name__ == '__main__':
    print('estimate score by known events')
    run_dbscan()

#     path_to_test = "../input/test"
#     test_dataset_submissions = []

#     create_submission = True  # True for submission
#     if create_submission:
#         print('process test events')
#         for event_id, hits in load_dataset(path_to_test, parts=['hits']):
#             print('Event ID: ', event_id)
#             labels = do_dbscan_predict(hits)
#             # Prepare submission for an event
#             one_submission = create_one_event_submission(event_id, hits['hit_id'].values, labels)
#             test_dataset_submissions.append(one_submission)

#         # Create submission file
#         submussion = pd.concat(test_dataset_submissions, axis=0)
#         submussion.to_csv('submission_final.csv', index=False)
# org: 0.42 (eps = 0.0045)
# after zrt: 0.41 (removed zrt)
# eps: 0.0048: 0.43233481315860445
# 180 angles: 0.441763680947588
# 360: 0.4337772013900576 (less!)
# z shift random - 0.06236001632828447
# zshift linspace: 0.441763680947588
# shifts = [-5.5, -5.27551020, -5.05102041, 5.05102041]: 0.4782573730101376
# 50 shits: 0.489
# 0.079861982990451 formula

estimate score by known events
[38.135510], z_s: 0.00000000, score : 0.08007820
[ 0] score : 0.08007820
--------------------------------------
0.08007819659883353


In [ ]:
# c > 3
# [46.136068], z_s: 0.00000000, score : 0.44176368
# [46.775323], z_s: -5.50000000, score : 0.38450144
# [-5.50000000] score : 0.45783910
# [45.040371], z_s: -5.27551020, score : 0.29081078
# [-5.27551020] score : 0.44779677

# c > 7
# 44.588105], z_s: 0.00000000, score : 0.44176368
# [45.355518], z_s: -5.50000000, score : 0.38450144
# [-5.50000000] score : 0.45489699
# [45.183360], z_s: -5.27551020, score : 0.29081078
# [-5.27551020] score : 0.44521397

In [ ]:
# estimate score by known events
# 199
# [49.976010], z_s: 0.00000000, score : 0.44176368
# 199
# [50.850307], z_s: -5.50000000, score : 0.38450144
# 199
# [48.037869], z_s: -5.27551020, score : 0.29081078
# 199
# [51.146998], z_s: -5.05102041, score : 0.18956037
# 199
# [51.095927], z_s: -4.82653061, score : 0.10928646
# 199
# [52.611142], z_s: -4.60204082, score : 0.06405998
# 199
# [54.109119], z_s: -4.37755102, score : 0.04192227
# 199
# [50.738167], z_s: -4.15306122, score : 0.02613579
# 199
# [52.260359], z_s: -3.92857143, score : 0.01964266
# 199
# [51.789074], z_s: -3.70408163, score : 0.01459217
# 199
# [52.848817], z_s: -3.47959184, score : 0.01212906
# 199
# [50.870826], z_s: -3.25510204, score : 0.01070783
# 199
# [50.235415], z_s: -3.03061224, score : 0.00909727
# 199
# [49.270647], z_s: -2.80612245, score : 0.00599853
# 199
# [49.129222], z_s: -2.58163265, score : 0.00649701
# 199
# [48.286129], z_s: -2.35714286, score : 0.00562762
# 199
# [49.842293], z_s: -2.13265306, score : 0.00491236
# 199
# [49.384661], z_s: -1.90816327, score : 0.00477814
# 199
# [48.717680], z_s: -1.68367347, score : 0.00457546
# 199
# [48.012776], z_s: -1.45918367, score : 0.00436190
# 199
# [49.221061], z_s: -1.23469388, score : 0.00420920
# 199
# [48.068072], z_s: -1.01020408, score : 0.00431361
# 199
# [47.772906], z_s: -0.78571429, score : 0.00423060
# 199
# [48.778291], z_s: -0.56122449, score : 0.00381379
# 199
# [47.947019], z_s: -0.33673469, score : 0.00366165
# 199
# [48.434124], z_s: -0.11224490, score : 0.00368642
# 199
# [48.716099], z_s: 0.11224490, score : 0.00366165
# 199
# [48.707970], z_s: 0.33673469, score : 0.00381379
# 199
# [48.511677], z_s: 0.56122449, score : 0.00423060
# 199
# [47.882633], z_s: 0.78571429, score : 0.00431361
# 199
# [49.330805], z_s: 1.01020408, score : 0.00420920
# 199
# [49.659249], z_s: 1.23469388, score : 0.00436190
# 199
# [49.121822], z_s: 1.45918367, score : 0.00457546
# 199
# [50.480902], z_s: 1.68367347, score : 0.00477814
# 199
# [50.040903], z_s: 1.90816327, score : 0.00491236
# 199
# [50.346628], z_s: 2.13265306, score : 0.00562762
# 199
# [50.045502], z_s: 2.35714286, score : 0.00649701
# 199
# [50.590774], z_s: 2.58163265, score : 0.00599853
# 199
# [53.998526], z_s: 2.80612245, score : 0.00909727
# 199
# [50.601889], z_s: 3.03061224, score : 0.01070783
# 199
# [50.619714], z_s: 3.25510204, score : 0.01212906
# 199
# [51.517029], z_s: 3.47959184, score : 0.01459217
# 199
# [49.824840], z_s: 3.70408163, score : 0.01964266
# 199
# [49.964861], z_s: 3.92857143, score : 0.02613579
# 199
# [50.102062], z_s: 4.15306122, score : 0.04192227
# 199
# [50.318057], z_s: 4.37755102, score : 0.06405998
# 199
# [51.581126], z_s: 4.60204082, score : 0.10928646
# 199
# [52.135404], z_s: 4.82653061, score : 0.18956037
# 199
# [52.178464], z_s: 5.05102041, score : 0.29081078
# 199
# [51.919961], z_s: 5.27551020, score : 0.38450144
# 199
# [48.914852], z_s: 5.50000000, score : 0.44176368
# [ 0] score : 0.43466177
# --------------------------------------
# 0.43466176909896603